### Points noted from SAA exam learning path (https://jayendrapatil.com/aws-solution-architect-associate-exam-learning-path/)

- Remember the use case of SWF and it's action tasker (used example of gmail percentage of storage left), its collaboration with DynamoDB + SES
- Remember the use case of any upload media website like FB (use of SIF + multi-part upload (has features like upload without complete knowledge of size)
- For RDS running on EC2 instances with EBS attached vol, use Provisioned IOPS EBS + EBS-Optimized Instance type, if RDS is bottlenecked
- __innoDB__ is the recommended storage engine for MySQL DB instances on RDS while __xtraDB__ storage engine for MariaDB
- AWS Config: shows what resources we have now; IAM Credentials Report: allows auditing credentials and login
- DR: AMI for EC2 backup; Snapshot for EBS Volume; asyncronous db backup on S3(slow service) or EBS or EC2 instance for RDS
- Keep rapidly accessed data to EBS/RDS which offer high IOPS rather than on S3
- Gateway-cached volumes utilizes S3 for primary data backup, while retaining frequently accessed data locally in a cache.
- Gateway-stored volumes stores the complete primary data locally, while asynchronously backing up that data to AWS (offsite backup)
---
**Gateway-Cached Volume**
- Data is stored in S3 and acts as a Primary data storage
- Gateway retains a copy of recently read data locally for low latency access to the frequently accessed data

**Gateway-Stored Volume**
- Gateway-stored volumes maintain the entire data set locally to provide low latency access
- Gateway asynchronously backs up point-in-time snapshots (in the form of EBS snapshots) of the data to S3 which provides durable off-site backups

> although data is stored in S3, gateway volumes are only accessible from the AWS Storage Gateway and cannot be directly accessed using Amazon S3 APIs

Q. Which of the following services natively encrypts data at rest within an AWS region?Choose 2 answers

__a. AWS Storage Gateway__
b. Amazon DynamoDB
c. Amazon CloudFront
__d. Amazon Glacier__
e. Amazon Simple Queue Service

---

![](./resources/AWS-VPC-Components.png)

---

- Route tables needs to be updated to defined routes for Internet gateways, Virtual Private gateways, VPC Peering, VPC Endpoints, NAT Device etc.
- ASG Launch configuration can’t be modified after creation and needs to be created new if any modification required; similarly vpc flowlogs
- Go through ASG Questions especially Q.34: https://jayendrapatil.com/aws-auto-scaling/
- detailed monitoring on EC2 will be enabled by the default launch configuration of Auto Scaling
- Auto Scaling Launch config does not cost anything
- GroupDesiredCapacity: The number of instances that the Auto Scaling group attempts to maintain
- SQS: Consumer should __explicitly delete__ the message from the Queue once it is received and successfully processed
- For situations where the database is not able to handle the amount of WRITE, use SQS
- RDS: too many reads, then use elastic cache in b/w; too many writes, use SQS in b/w
- If you create SQS queue and do not perform any operation on it like send or receive msg, AWS will delete the queue after __30 days without any notification__

![](./resources/sqs_arch.jpg)

- ELB with TCP listener and proxy protocol will allow IP to be passed 
- ELB with HTTPs does not support Client-Side certificates
- ELB and EC2 must be in the same VPC
- The user can add more EC2 instances or more zones (to increase HA) to ELB __on the fly__
- IAM Policy: IAM policy basically states __“Principal A is allowed or denied (effect) to perform Action B on Resource C given Conditions D are satisfied”__
- One AWS account can have 5k users, 250 roles and 100 groups and 1 MFA device in use (per account)
- You have multiple departments in your organization: Create IAM groups based on the permission and assign IAM users to the groups
- Best practices to secure EC2 instance:
    - Keep up to date with software patches (apt update)
    - Lock down ports to prevent unauthorized access (SG e.g allow just https, http and ssh)
    - Ditch passwords and require administrators to use SSH keys
    
__Cross-Account Access:__
- AWS allows granting __cross-account access__ to AWS resources, which can be done using IAM __Roles or Resource Based policies__
1.  __Resource Based Policy:__
    - Resource based policy allows you to attach a policy directly to the resource that you want to share, instead of using a role as a proxy.
    - Resource-based policy specifies who, as a Principal in the form of a list of AWS account ID numbers, can access that resource and what they can access __(authentication and authorization)__
---
2. __IAM Role:__
- Roles can be assumed by
1. IAM user within same AWS account
2. IAM user from different AWS account
3. AWS services such as EC2, RDS, EMR to interact with other services
4. An external user authenticated by external identity provider (IdP) (WIF/Cognito)

- Roles involve 2 defining policies: (for cross-account access)
1. __Trust Policy__ (authentication)
    - It defines who can assume the role
    - It involves setting up trust b/w account who owns the resource and the account who owns the user that needs access to the resource
2. __Permission/Access Policy__ (authorization)
    - It determines authorization i.e what they (user of the role) can access

Example:
    - An IAM trust policy allows the _EC2 instance to assume an EC2 instance role_
    - An IAM access policy allows the EC2 role to access S3 objects


---

### VPC

- NAT Gateway enables instances in a private subnet to connect to the internet (for example, for software updates) or other AWS services, but prevent the internet from initiating connections with the instances.
- NAT devices are not supported for IPv6 traffic, use an Egress-only Internet gateway instead
- Security group restricts access to EC2 while ACL restricts traffic to a subnet
- Bastion host:
    - Deployed on public subnet
    - can be used for RDP (windows) and ssh (Linux)

> Summary: NAT instances, Egress-only IGW, Bation host

---

### ELB

- ELB does not support __client side certificate__ for HTTPS listener. For client side certificate support, use TCP for connection b/w client and ELB with SSL protocol
- Uses __Server Name Indication__ for mulitple SSL certificates
- supports SSL termination, to decrypt the request on ALB before sending it to the underlying targets
- only NLB supports Elastic IP
- NLB does not support __sticky session__

### ASG

- can't span multiple regions just like AMIs (ENI can't span multiple AZs, EBS can't span AZs; 1 NAT gateway in 1 AZ); SG can't span VPCs

> Reminder: connection b/w corporate datacenter and VPC can be done via private subnet as it doesn't require public IP address

### EC2

- Instance Types: TRC, MIG
- CloudWatch alarms can do a lot of things for EC2 like stop, terminate, reboot instance etc. (for this the IAM USER has to have cloudwatch access as well as ec2 rights)
- Aggregation of data (cloudwatch) for mulitple ec2 instances can only be done with detailed monitoring
- for system status check impaired status you need Stop Start for EBS and terminate and relaunch for Instance store
- Troubleshoot:
    - instance gets terminated when launching: could be that account has reached max instance limit
    - while launching instance shows "insufficient instance capacity": could be that AWS does not have sufficient capacity in that AZ
    - varioud problems while connecting via SSH: i) chmod 400 .pem (unprotected error for key) ii) host key not found (wrong username) iii) connection timed out (private key file is proper)
- Recommendation: Do not select the AZ; instead let AWS select the AZ
- If you stop an instance, modify the user data, and start the instance, the new user data is not executed automatically

### ENI

- Enhanced Networking works only in VPC
- ENI: cold attach (during launch process), warm attach (when ec2 stopped), hot attach (when ec2 running)
- when EC2 is terminated, ENI attached to it is also terminated which means the instance does not retain its private IP but when instance is stopped/restarted it retains its private IP and thus to ensure failover capabilities some ppl give secondary private ip for incomming traffic (but not necessary)
- AWS cannot assign public IPs for instance with multiple ENIs (A user is planning to host a web server as well as an app server on a single EC2 instance, which is a part of the public subnet of a VPC. How can the user setup to have two separate public IPs and separate security groups for both the application as well as the web server? __Launch a VPC instance with two network interfaces. Assign a separate security group and elastic IP to them.__)

### SG

- Security groups are associated with network interfaces
- If multiple rules are defined for the same protocol and port, the Most permissive rule is applied for e.g. for multiple rules for tcp and port 22 for specific IP and Everyone, everyone is granted access being the most permissive rule

### EBS

- The user will be charged for the volume even if the instance is stopped
- A user has launched an EBS backed instance. The user started the instance at 9 AM in the morning. Between 9 AM to 10 AM, the user is testing some script. Thus, he stopped the instance twice and restarted it. In the same hour the user rebooted the instance once. For how many instance hours will AWS charge the user? __3 hours; For restart it does not charge for an extra hour, while every stop/start it will be charged as a separate/additional hour__
- EBS-backed AMIs launch faster than EC2 instance store-backed AMIs
- A user is trying to create a PIOPS EBS volume with 4000 IOPS and 100 GB size. AWS does not allow the user to create this volume. What is the possible root cause for this?
The ratio between IOPS and the EBS volume is higher than 30

### S3

- Pre-signed URLs allows user to be able to download or upload a specific object without requiring AWS security credentials or permissions
- Pre-signed URLs Question Takeaways:
    - they can expire
    - application not using valid credentials to generate pre-signed url
- Best Practices:
    - use parallel threads and __Multipart upload__ for faster writes
    - use parallel threads and __Range Header GET__ for faster reads
    - for list operations with large number of objects, its better to build a secondary index in DynamoDB
    - use Versioning to protect from unintended overwrites and deletions, but this does not protect against bucket deletion
    - use VPC S3 Endpoints with VPC to transfer data using Amazon internal network
- Bucket policy can be used to grant cross-account access to other AWS accounts or IAM users in other accounts for the bucket and objects in it.
- Bucket policy can also be used to restrict IP Address to access bucket
- Object and bucket can have ACL but folders cannot have ACL
- MFA Delete can be enabled on a bucket to ensure that data in the bucket cannot be accidentally deleted
- While the bucket owner, the AWS account that created the bucket (root account), and all authorized IAM users can enable versioning, but only the bucket owner (root account) can enable MFA delete.
- MFA Delete however does not prevent deletion or allows restoration.

### CloudFront

- Security
    - __Origin Access Identity (OAI)__ can be used to restrict the content from S3 origin to be accessible from CloudFront only. Origin Access Identity is the CloudFront special user which allows access to the resources only through CloudFront while denying direct access to the origin content for e.g. if S3 is the Origin for CloudFront, S3 can be configured to allow access only from OAI and hence deny direct access
    - supports __Geo restriction (Geo-Blocking) to whitelist or blacklist countries__ that can access the content
    
- only caches responses to GET and HEAD requests and, optionally, OPTIONS requests
- does not cache responses to PUT, POST, PATCH, DELETE request methods and these requests are proxied back to the origin

---

### Secure Architecture

![](./resources/aws_secure.png)

### RDS

- Multi-AZ deployment for the DB instance reduces the impact of a maintenance event by following these steps:
    - Perform maintenance on the standby
    - Promote the standby to primary
    - Perform maintenance on the old primary, which becomes the new standby
- If you give maintenance window, then the RDS then go __offline__ if there is __security patching__ while updation
- If you have not given maintenance window and if you later on change the RDS instance size from micro to large and want the scale up to happen during maintenance window then AWS will choose to do it in the __default maintenance window__.
- DB instance can be modified to use additional storage and converted to a different storage type.
- However, storage allocated for a DB instance __cannot be decreased__. MySQL, MariaDB, PostgreSQL, and Oracle DB instances can be scaled up for storage, which helps improve I/O capacity.
- Storage capacity nor the type of storage __for a SQL Server__ DB instance can't be changed due to extensibility limitations of striped storage attached to a Windows Server environment.

### ELB

- CLB and ALB support idle connection timeout while NLB not
- CLB and ALB support sticky session while NLB not
- NLB support Elastic IP while others not
- Classic ELB and ALB do not preserve the client side source IP. It needs to be retrieved using X-Forward-Header or proxy protocol
- Enable __access logs__ on laod balancer to capture connection information
- connection draining (deregistration delay) has 300 seconds default draining time and 3600 seconds as max
- HA and redundancy of ELB is achieved by Route53 (e.g Failover routing policy)

### ASG

- When creating ASG, if you don't add ELB health check then the unhealthy instance will not be terminated by ASG
- When creating ASG, if you suspend "AddToLoadBalancer" then the new instance will not be registered to ELB and the instance has to be manually registered ever after removing the suspension.
- Launch configuration can't be modified after creation and needs to be created new if modification required

- GAG (gateway and glacier): Gateway (storage gateway) and Glacier __natively encrypts__ data at rest